In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


### Create the Search Service

In [ ]:
-- Create a stage for docs

CREATE STAGE if not exists agent_materials 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

In [ ]:
MY_STAGE = 'agent_materials'
MY_FILE_NAME = "snowflake_10k_2025.pdf"

# Upload the file to a stage.
put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)
put_result[0].status

In [ ]:
CREATE OR REPLACE TABLE raw_text_10k AS
SELECT
    RELATIVE_PATH,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@agent_materials',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'} ):content
        ) AS EXTRACTED_LAYOUT
FROM
    DIRECTORY('@agent_materials')
WHERE
    RELATIVE_PATH LIKE '%.pdf';

In [ ]:
CREATE OR REPLACE TABLE doc_chunks_10k AS
SELECT
    relative_path,
    BUILD_SCOPED_FILE_URL(@docs, relative_path) AS file_url,
    CONCAT(relative_path, ': ', c.value::TEXT) AS chunk,
    'English' AS language
FROM
    raw_text_10k,
    LATERAL FLATTEN(SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
        EXTRACTED_LAYOUT,
        'markdown',
        2000, -- chunks of 2000 characters
        300 -- 300 character overlap
    )) c;

In [ ]:
# This cell will give you your current warehouse
# Replace line 4 in the cell below with your warehouse

current_warehouse = session.get_current_warehouse()
print(current_warehouse)

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE snow_10k_2025
    ON chunk
    ATTRIBUTES language
    WAREHOUSE = wh_xs
    TARGET_LAG = '1 day'
    AS (
    SELECT
        chunk,
        relative_path,
        file_url,
        language
    FROM doc_chunks_10k
    );

Create Table for cortex analyst

In [ ]:
MY_FILE_NAME = "SP500.csv"

# Upload the file to a stage.
put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)
put_result[0].status

In [ ]:
CREATE OR REPLACE TABLE SP_500 ( EXCHANGE VARCHAR , SYMBOL VARCHAR , SHORTNAME VARCHAR , LONGNAME VARCHAR , SECTOR VARCHAR , INDUSTRY VARCHAR , CURRENTPRICE NUMBER(38, 3) , MARKETCAP NUMBER(38, 0) , EBITDA NUMBER(38, 0) , REVENUEGROWTH NUMBER(38, 3) , CITY VARCHAR , STATE VARCHAR , COUNTRY VARCHAR , FULLTIMEEMPLOYEES NUMBER(38, 0) , LONGBUSINESSSUMMARY VARCHAR , WEIGHT NUMBER(38, 20) ); 

CREATE OR REPLACE TEMP FILE FORMAT SP_LOAD
	TYPE=CSV
    SKIP_HEADER=1
    FIELD_DELIMITER=','
    TRIM_SPACE=TRUE
    FIELD_OPTIONALLY_ENCLOSED_BY='"'
    REPLACE_INVALID_CHARACTERS=TRUE
    DATE_FORMAT=AUTO
    TIME_FORMAT=AUTO
    TIMESTAMP_FORMAT=AUTO; 

COPY INTO SP_500
FROM (SELECT $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12, $13, $14, $15, $16
	FROM '@AGENT_MATERIALS') 
FILES = ('SP500.csv') 
FILE_FORMAT = 'SP_LOAD' 
ON_ERROR=ABORT_STATEMENT 

Now that we have the Search Service and the table for Cortex Analyst all we have left to do is create the Analyst Semantic View.  

You can use the yaml provided or try it out yourself in the UI by navigating to Cortex Analyst under AI/ML and building it yourself

In [ ]:
MY_FILE_NAME = "SP500.yaml"

# Upload the file to a stage.
put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)
put_result[0].status

In [ ]:
MY_FILE_NAME = "agent_gateway.zip"

# Upload the file to a stage.
put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)
put_result[0].status

Open up Streamlit in Snowflake
Create a new app on an XS Warehouse
Once in the app add the agent_gateway.zip file to Staged Packages in the top right corner
in the packages in the top left add aiohttp, pydantic


We just uploaded the zip file to the stage.  My stage is in CROMANO.DEMO so when it ask for path replace my Database and Schema with yours.  For example my location is @CROMANO.DEMO.AGENT_MATERIALS/agent_gateway.zip